## Домашнее задание к занятию "A/B-тесты"

### Описание задачи

![banner](https://storage.googleapis.com/kaggle-datasets-images/635/1204/126be74882028aac7241553cef0e27a7/dataset-original.jpg)

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) И относятся к одному или двум так называемым классам (вода, огонь и т. д.).
Профессор Оук является изобретателем Pokedex - портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание набора данных
Профессор Оук скопировал все содержимое в память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* `pid`: Numeric - ID покемона
* `HP`: Numeric - Очки здоровья
* `Attack`: Numeric - Сила обычной атаки
* `Defense`: Numeric - Сила обычной защиты
* `Sp. Atk`: Numeric - Сила специальной атаки
* `Sp. Def`: Numeric - Сила специальной защиты
* `Speed`: Numeric - Скорость движений
* `Legendary`: Boolean - «True», если покемон редкий
* `Class 1`: Categorical - Класс покемона
* `Class 2`: Categorical - Класс покемона

In [1]:
import warnings
# Отключение предупреждений (warnings)
warnings.filterwarnings("ignore")

import pandas as pd
import scipy.stats as stats

from scipy.stats import ttest_ind
from scipy.stats import f_oneway, shapiro

pokemon = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/pokemon.csv', on_bad_lines='skip')  # Откроем датасет
pokemon.head()

# Обратите внимание, что у покемона может быть один или два класса.
# Если у покемона два класса, считается, что они имеют одинаковую значимость.

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [15]:
classes = pokemon['Class 1'].unique()
print(classes)


['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']


In [16]:
classes = pokemon['Class 2'].unique()
print(classes)

['Poison' nan 'Flying' 'Dragon' 'Ground' 'Fairy' 'Grass' 'Fighting'
 'Psychic' 'Steel' 'Ice' 'Rock' 'Dark' 'Water' 'Electric' 'Fire' 'Ghost'
 'Bug' 'Normal']


In [2]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pid        800 non-null    int64 
 1   Name       799 non-null    object
 2   Class 1    800 non-null    object
 3   Class 2    414 non-null    object
 4   HP         800 non-null    int64 
 5   Attack     800 non-null    int64 
 6   Defense    800 non-null    int64 
 7   Sp. Atk    800 non-null    int64 
 8   Sp. Def    800 non-null    int64 
 9   Speed      800 non-null    int64 
 10  Legendary  800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 63.4+ KB


### Задачи

<div class="alert alert-info">
<b>Задание № 1:</b>
    
Профессор Оук подозревает, что покемоны в классе `Grass` имеют более сильную обычную атаку, чем покемоны в классе `Rock`. Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
    
Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;
    
Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

</div>

In [3]:
grass_data = pokemon[pokemon['Class 1'] == 'Grass'].copy()
rock_data = pokemon[pokemon['Class 1'] == 'Rock'].copy()

# Удаление покемонов, которые относятся к обоим классам
grass_data.drop(grass_data[grass_data['Class 2'] == 'Rock'].index, inplace=True)
rock_data.drop(rock_data[rock_data['Class 2'] == 'Grass'].index, inplace=True)

# A/B-тестирование
statistic, p = stats.ttest_ind(grass_data['Attack'], rock_data['Attack'])
print('Задание 1:')
print('t-статистика:', round(statistic,5))
print('p-value:',round(p, 5))

if p < 0.05:
    print("Покемоны класса Grass имеют более сильную обычную атаку, чем покемоны класса Rock.")
else:
    print("Нет статистически значимых различий в обычной атаке между классами Grass и Rock.")

Задание 1:
t-статистика: -3.68673
p-value: 0.00035
Покемоны класса Grass имеют более сильную обычную атаку, чем покемоны класса Rock.


<div class="alert alert-info">
<b>Задание № 2:</b>
    
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос, а правда ли, что покемоны в классе `Water` в среднем быстрее, чем покемоны в классе `Normal`.
    
    
Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.
</div>

In [4]:
water_data = pokemon[pokemon['Class 1'] == 'Water'].copy()
normal_data = pokemon[pokemon['Class 1'] == 'Normal'].copy()

# Удаление покемонов, которые относятся к обоим классам
water_data.drop(water_data[water_data['Class 2'] == 'Normal'].index, inplace=True)
normal_data.drop(normal_data[normal_data['Class 2'] == 'Water'].index, inplace=True)

# A/B-тестирование
statistic, p = stats.ttest_ind(water_data['Speed'], normal_data['Speed'])
print('t-статистика:', round(statistic,5))
print('p-value:',round(p, 5))

if p < 0.05:
    print("Покемоны класса Water в среднем быстрее, чем покемоны класса Normal.")
else:
    print("Нет статистически значимых различий в скорости движения между классами Water и Normal.")

t-статистика: -1.56678
p-value: 0.11869
Нет статистически значимых различий в скорости движения между классами Water и Normal.


<div class="alert alert-info">
<b>Задание № 3:</b>
    
Профессор Оук тот еще безумец. Он изобрёл сыворотку, способную ускорить покемона. Однако мы усомнились в эффективности его вакцины. Професоор дал эту сыворотку следующим покемонам: смотри массив `treathed_pokemon`. Проверьте, работает ли вообще его сыворотка, убедите всех в своём выводе статистически.
    
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

</div>

In [5]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill', 'Mega Alakazam',
                    'Deoxys Normal Forme', 'Mega Lopunny']

In [6]:


treated_data = pokemon[pokemon['Name'].isin(treathed_pokemon)].copy()

# A/B-тестирование сравнения скорости обработанных покемонов с остальными
untreated_data = pokemon[~pokemon['Name'].isin(treathed_pokemon)].copy()
statistic, p = stats.ttest_ind(treated_data['Speed'], untreated_data['Speed'])

print('t-статистика:', round(statistic,5))
print('p-value:',round(p, 5))


if p < 0.05:
    print("Сыворотка профессора Оука повышает скорость покемонов.")
else:
    print("Нет статистически значимых различий в скорости покемонов, получивших сыворотку.")

t-статистика: 5.38541
p-value: 0.0
Сыворотка профессора Оука повышает скорость покемонов.


<div class="alert alert-info">
<b>Задание № 4:</b>
    
Профессор Оук всегда любил истории про легендарных покемонов. Однако профессор не очень уверен, что они лучше остальных покемонов. Оук предложил разобраться в этом нам. Проверьте, действительно ли сумма характеристик `HP`,`Attack`,`Defense` у легендарных покемонов выше, чем у других покемонов?

А произведение этих же параметров?

Найдите ответы на эти вопросы и убедите всех в своём выводе статистически.
   

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

</div>

In [7]:
legendary_data = pokemon[pokemon['Legendary'] == True].copy()
non_legendary_data = pokemon[pokemon['Legendary'] == False].copy()

# A/B-тестирование для суммы и произведения характеристик у легендарных и не легендарных покемонов
statistic_sum, p_sum = stats.ttest_ind(legendary_data['HP'] + legendary_data['Attack'] + legendary_data['Defense'],
                                               non_legendary_data['HP'] + non_legendary_data['Attack'] + non_legendary_data['Defense'])
statistic_prod, p_prod = stats.ttest_ind(legendary_data['HP'] * legendary_data['Attack'] * legendary_data['Defense'],
                                                 non_legendary_data['HP'] * non_legendary_data['Attack'] * non_legendary_data['Defense'])


print('t-статистика (сумма):', round(statistic_sum,5))
print('p-value (сумма):', round(p_sum,5))
print('t-статистика (произведение):', round(statistic_prod,5))
print('p-value (произведение):',round(p_prod,5))

if p_sum < 0.05:
    print("Сумма характеристик HP, Attack и Defense у легендарных покемонов выше, чем у других покемонов.")
else:
    print("Нет статистически значимых различий в сумме характеристик у легендарных и других покемонов.")
print('t-статистика (произведение):', round(statistic_prod,5))
print('p-value (произведение):',round(p_prod,5))
if p_prod < 0.05:
    print("Произведение характеристик HP, Attack и Defense у легендарных покемонов выше, чем у других покемонов.")
else:
    print("Нет статистически значимых различий в произведении характеристик у легендарных и других покемонов.")

t-статистика (сумма): 11.59185
p-value (сумма): 0.0
t-статистика (произведение): 13.26325
p-value (произведение): 0.0
Сумма характеристик HP, Attack и Defense у легендарных покемонов выше, чем у других покемонов.
t-статистика (произведение): 13.26325
p-value (произведение): 0.0
Произведение характеристик HP, Attack и Defense у легендарных покемонов выше, чем у других покемонов.


<div class="alert alert-info">
<b>Задание № 5:</b>
    
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боёв Оук выделил четыре класса `best_defence_class`, которые на его взгляд одинаковы по "силе обычной защиты" `Defense`.

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Всё та же статистика вам в помощь!
   

Вы можете предположить, что распределение параметров защитных характеристик является нормальным для всех классов покемонов.

</div>

In [8]:
best_defence_class = ['Rock', 'Ground', 'Steel', 'Ice']
best_defence_class

['Rock', 'Ground', 'Steel', 'Ice']

In [9]:
best_defence_classes = ["Class A", "Class B", "Class C", "Class D"]  # Список классов с одинаковым уровнем защиты

defense_data = pokemon[pokemon['Class 1'].isin(best_defence_class)].copy()

# A/B-тестирование для сравнения уровня защиты между классами
statistic, p = stats.f_oneway(*[defense_data[defense_data['Class 1'] == c]['Defense'] for c in best_defence_class])

print('t-статистика:', round(statistic,5))
print('p-value:', round(p,5))

if p < 0.05:
    print("Уровень защиты классов покемонов статистически отличается.")
else:
    print("Нет статистически значимых различий в уровне защиты между классами покемонов.")

t-статистика: 10.57688
p-value: 0.0
Уровень защиты классов покемонов статистически отличается.


# **Примечание:**

Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/). Мы не сможем проверить его или помочь, если вы пришлете:

*   файлы;
*   архивы;
*   скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в Discord.

**Как правильно задавать вопросы аспирантам, преподавателям и коллегам:**

Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:

1.   Что я делаю?
2.   Какого результата я ожидаю?
3.   Как фактический результат отличается от ожидаемого?
4.   Что я уже попробовал сделать, чтобы исправить проблему?

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.

In [19]:
meangroup1 = pokemon[pokemon['Class 1'].isin(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
                                              'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
                                              'Dragon', 'Dark', 'Steel', 'Flying'])].copy()

meangroup2 = pokemon[pokemon['Class 1'].isin(['Poison', 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass', 'Fighting',
                                              'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water', 'Electric', 'Fire',
                                              'Ghost', 'Bug', 'Normal'])].copy()



In [21]:
statistic, p = stats.ttest_ind(meangroup1['Attack'], meangroup2['Attack'])
print('t-статистика:', round(statistic,5))
print('p-value:',round(p, 5))

if p < 0.05:
    print("Среднее значение в группе meangroup1 больше, чем в группе meangroup2.")
else:
    print("Нет статистически значимых различий в среднем значении между группами meangroup1 и meangroup2.")


t-статистика: 0.0
p-value: 1.0
Нет статистически значимых различий в среднем значении между группами meangroup1 и meangroup2.
